In [17]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation,BatchNormalization, Flatten, Conv2D, MaxPooling2D
from keras.models import Model
from keras.preprocessing import image
from keras import losses
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os
from sklearn.metrics import log_loss
from glob import glob

In [57]:
#Parsing all the t
path = '/kaggle/input/csc4851-homework4/birds_400/train/'
train_datagen = ImageDataGenerator(rescale=1. / 224)
train = train_datagen.flow_from_directory(path, target_size=(224,224), class_mode='categorical')

classes = [x[0].split('/')[-1] for x in os.walk(path)]
classes.pop(0)
print((classes))

Found 58388 images belonging to 400 classes.
['WATTLED CURASSOW', 'CARMINE BEE-EATER', 'GAMBELS QUAIL', 'UMBRELLA BIRD', 'AMERICAN KESTREL', 'AMERICAN GOLDFINCH', 'DUSKY LORY', 'BLACK THROATED WARBLER', 'YELLOW CACIQUE', 'STRIPPED SWALLOW', 'VERMILION FLYCATHER', 'CAPE MAY WARBLER', 'RED TAILED HAWK', 'GURNEYS PITTA', 'INDIAN ROLLER', 'AZURE TANAGER', 'GREEN MAGPIE', 'BANDED BROADBILL', 'GREY PLOVER', 'TOUCHAN', 'HYACINTH MACAW', 'HARLEQUIN DUCK', 'ABYSSINIAN GROUND HORNBILL', 'CALIFORNIA QUAIL', 'BALD IBIS', 'RUBY THROATED HUMMINGBIRD', 'INDIGO BUNTING', 'ALBERTS TOWHEE', 'SORA', 'HOODED MERGANSER', 'CAPE ROCK THRUSH', 'BLUE COAU', 'CALIFORNIA CONDOR', 'BORNEAN PHEASANT', 'BARN SWALLOW', 'TAIWAN MAGPIE', 'SPLENDID WREN', 'BLACK THROATED BUSHTIT', 'CHINESE POND HERON', 'AFRICAN EMERALD CUCKOO', 'COMMON LOON', 'COMMON POORWILL', 'FIORDLAND PENGUIN', 'SUPERB STARLING', 'CUBAN TODY', 'TASMANIAN HEN', 'BLACK SWAN', 'WATTLED LAPWING', 'RED NAPED TROGON', 'AMERICAN PIPIT', 'EASTERN TOWEE', '

In [19]:
# Using alexnet
def AlexNet(input_shape):
    
    X_input = Input(input_shape)
    
    X = Conv2D(96,(11,11),strides = 4,name="conv0")(X_input)
    X = BatchNormalization(axis = 3 , name = "bn0")(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max0')(X)
    
    X = Conv2D(256,(5,5),padding = 'same' , name = 'conv1')(X)
    X = BatchNormalization(axis = 3 ,name='bn1')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max1')(X)
    
    X = Conv2D(384, (3,3) , padding = 'same' , name='conv2')(X)
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(384, (3,3) , padding = 'same' , name='conv3')(X)
    X = BatchNormalization(axis = 3, name = 'bn3')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(256, (3,3) , padding = 'same' , name='conv4')(X)
    X = BatchNormalization(axis = 3, name = 'bn4')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max2')(X)
    
    X = Flatten()(X)
    
    X = Dense(4096, activation = 'relu', name = "fc0")(X)
    
    X = Dense(4096, activation = 'relu', name = 'fc1')(X) 
    
    X = Dense(400,activation='softmax',name = 'fc2')(X)
    model = Model(inputs = X_input, outputs = X, name='AlexNet')
    
    return model

In [20]:
alex = AlexNet(train[0][0].shape[1:])

In [21]:
alex.summary()

Model: "AlexNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv0 (Conv2D)               (None, 54, 54, 96)        34944     
_________________________________________________________________
bn0 (BatchNormalization)     (None, 54, 54, 96)        384       
_________________________________________________________________
activation_5 (Activation)    (None, 54, 54, 96)        0         
_________________________________________________________________
max0 (MaxPooling2D)          (None, 26, 26, 96)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 26, 26, 256)       614656    
_________________________________________________________________
bn1 (BatchNormalization)     (None, 26, 26, 256)       1024

In [22]:
alex.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics=['accuracy'])

In [23]:
alex.fit_generator(train,epochs=5)

Epoch 1/5
1825/1825 [==============================] - 155s 85ms/step - loss: 5.8330 - accuracy: 0.0074
Epoch 2/5
1825/1825 [==============================] - 137s 75ms/step - loss: 5.5152 - accuracy: 0.0093
Epoch 3/5
1825/1825 [==============================] - 137s 75ms/step - loss: 5.4671 - accuracy: 0.0099
Epoch 4/5
1825/1825 [==============================] - 136s 74ms/step - loss: 5.4099 - accuracy: 0.0101
Epoch 5/5
1825/1825 [==============================] - 143s 78ms/step - loss: 5.3681 - accuracy: 0.0120


In [24]:
path_test = '/kaggle/input/csc4851-homework4/birds_400/test'
test_datagen = ImageDataGenerator(rescale=1. / 224)
test = test_datagen.flow_from_directory(path_test, target_size=(224,224), class_mode='categorical')

Found 2000 images belonging to 400 classes.


In [25]:
preds = alex.evaluate_generator(test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]*100))

Loss = 5.393240451812744
Test Accuracy = 0.8500000461935997


In [26]:
#validation

path_test = '/kaggle/input/csc4851-homework4/birds_400/valid'
validation_datagen = ImageDataGenerator(rescale=1. / 224)
validation = validation_datagen.flow_from_directory(path_test, target_size=(224,224), batch_size = 1,class_mode='categorical')

Found 2000 images belonging to 400 classes.


In [27]:
preds = alex.evaluate_generator(validation)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]*100))

Loss = 5.384179592132568
Test Accuracy = 1.1500000022351742


In [32]:
def prepare(img_path):
    img = image.load_img(img_path, target_size=(224,224))
    x = image.img_to_array(img)
    x = x/224
    return np.expand_dims(x, axis=0)

In [61]:
from keras import losses



In [62]:
cce = losses.CategoricalCrossentropy()

In [67]:
# result = alex.predict([prepare("/kaggle/input/csc4851-homework4/birds_400/test/ABBOTTS BABBLER/1.jpg")])
# print(len(result[0]))
template = [0]*400
log_loss = {}
test_images = glob("/kaggle/input/csc4851-homework4/birds_400/test/" + "*/*.jpg")

for path in test_images:
    result = alex.predict([prepare(path)])
    actual_class = path.split('/')[-2]
    actual_class = "BLACK & YELLOW  BROADBILL" if actual_class == "BLACK & YELLOW BROADBILL" else actual_class
    actual_index = classes.index(actual_class)
    template[actual_index] = 1
    log_loss_current = cce(template, result[0]).numpy()
#     print(log_loss_current)
    if actual_index in log_loss:
        log_loss[actual_index] += (log_loss_current)/100
    else:
        log_loss[actual_index] = (log_loss_current)/100
    template[actual_index] = 0

# print(template)
# print(classes[int(x)])

In [82]:
print('Generating submission.csv file...')
np.savetxt(
    'submission.csv',
    np.rec.fromarrays([ids, values]),
    fmt=['%s', '%d'],
    delimiter=',',
    header='id,birds',
)

# # Look at the first few predictions
!head submission.csv

Generating submission.csv file...
# id,birds
0,3
1,7
2,2
3,2
4,3
5,2
6,4
7,4
8,4
